In [ ]:
import pandas as pd
import numpy as np

def drop_dup(_df):
    dup_index = _df[_df.index.duplicated()].index
    return _df.drop(dup_index)['PM2.5']

df_org = pd.read_csv(
    'http://api.khu-cpfd.com:9019/v1/logs/file/particle?method=mean&machine=120,134,107,199,121,124',
    index_col='DATE',
    parse_dates=True
)
df_org.index = df_org.index.floor('T')

df_107 = df_org[df_org['MACHINE'] == 107]
df_107 = drop_dup(df_107)

df_120 = df_org[df_org['MACHINE'] == 120]
df_120 = drop_dup(df_120)

df_121 = df_org[df_org['MACHINE'] == 121]
df_121 = drop_dup(df_121)

df_124 = df_org[df_org['MACHINE'] == 124]
df_124 = drop_dup(df_124)

df_134 = df_org[df_org['MACHINE'] == 134]
df_134 = drop_dup(df_134)

df_199 = df_org[df_org['MACHINE'] == 199]
df_199 = drop_dup(df_199)

df = pd.concat([df_107, df_120, df_121, df_124, df_134, df_199], axis=1).fillna(0)
df.columns = ['m107', 'm120', 'm121', 'm124', 'm134', 'm199']

In [ ]:
machine_indices = [[134, 199, 121], [120, 107, 124]]

trimed_df = df[(df.index > pd.to_datetime('2022-06-24 22:00')) & (df.index < pd.to_datetime('2022-06-25 10:00'))]

data = np.zeros((len(trimed_df), 2, 3))
for r in range(2):
    for c in range(3):
        m = f'm{machine_indices[r][c]}'
        data[:, r, c] = trimed_df[m].values

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc

plt.rcParams["figure.autolayout"] = True
plt.rcParams["animation.html"] = "jshtml"

vmin = data.max()
vmax = data.min()
fig, ax = plt.subplots(figsize=(14, 6))

dimension = (2, 3)
cmap = plt.get_cmap('binary')
plot = plt.matshow(data[0], cmap=cmap, fignum=0)
plt.colorbar()
plt.clim(vmin, vmax)

def animate(i):
    plot.set_data(data[i])
    return plot

anim = animation.FuncAnimation(fig, animate, interval=300, frames=len(data), repeat=False)